Foreign Function Interface
====

Wrapping functions written in C
----

### Steps 

- Write the C header and implemntation files
- Write the Cython `.pxd` file to decalre C function signatures
- Write the Cython `.pyx` file to wrap the C functions for Python
- Write `setup.py` to automate buiding of the Python extension module
- Run `python setup.py build_ext --inplace` to build the module
- Import module in Python like any other Python module

### C header file

In [1]:
%%file c_math.h

#pragma once
double plus(double a, double b);
double mult(double a, double b);
double square(double a);
double acc(double *xs, int size);

Overwriting c_math.h


### C implementation file

In [2]:
%%file c_math.c
#include <math.h>
#include "c_math.h"

double plus(double a, double b) {
    return a + b;
};

double mult(double a, double b) {
    return a * b;
};

double square(double a) {
    return pow(a, 2);
};

double acc(double *xs, int size) {
    double s = 0;
    for (int i=0; i<size; i++) {
        s += xs[i];
    }
    return s;
};

Overwriting c_math.c


### Cython "header" file

The `.pxd` file is similar to a header file for Cython. In ohter words, we can `cimport <filename>.pxd` in the regular Cython `.pyx` files to get access to functions decalred in the `.pxd` files.

In [3]:
%%file cy_math.pxd

cdef extern from "c_math.h":
    double plus(double a, double b)
    double mult(double a, double b)
    double square(double a)
    double acc(double *xs, int size)

Overwriting cy_math.pxd


### Cython "implementation" file

Here is whhere we actaully wrap the C code for use in Python. Note especially how we handle passing in of arrays to a C funciton expecting a pointer to double using `typed memoryviews`.

In [4]:
%%file cy_math.pyx

cimport cy_math

def py_plus(double a, double b):
    return cy_math.plus(a, b)

def py_mult(double a, double b):
    return cy_math.mult(a, b)

def py_square(double a):
    return cy_math.square(a)

def py_sum(double[::1] xs):
    return cy_math.acc(&xs[0], len(xs))

Overwriting cy_math.pyx


### Build script `setup.py`

This is build script for Python, similar to a Makefile

In [5]:
%%file setup.py

from distutils.core import setup, Extension
from Cython.Build import cythonize
import numpy as np

ext = Extension("cy_math",
                sources=["cy_math.pyx", "c_math.c"],
                libraries=["m"],
                extra_compile_args=["-w"])

setup(name = "Math Funcs",
      ext_modules = cythonize(ext))

Overwriting setup.py


### Building an extension module

In [6]:
! python setup.py clean
! python setup.py -q build_ext --inplace

Compiling cy_math.pyx because it changed.
[1/1] Cythonizing cy_math.pyx
running clean


### Using the extension module in Python

In [7]:
import cy_math

print(cy_math.py_plus(3, 4))
print(cy_math.py_mult(3, 4))
print(cy_math.py_square(3))

xs = np.arange(10, dtype='float')
print(cy_math.py_sum(xs))

7.0
12.0
9.0
45.0


### Confirm that we are geting C speedups by comparing with pure Python accumulator

In [8]:
def acc(xs):
    s = 0
    for x in xs:
        s += x
    return s

In [9]:
xs = np.arange(1000000, dtype='float')
%timeit acc(xs)
%timeit cy_math.py_sum(xs)

1 loops, best of 3: 200 ms per loop
100 loops, best of 3: 2.1 ms per loop


Wrap an R function from libRmath¶
----

R comes with a standalone C library of special functions and distributions, as described in the official documentation. These functions can be wrapped for use in Python.

In [10]:
%%file rmath.pxd

cdef extern from "Rmath.h":
    double  rnorm(double, double)

Overwriting rmath.pxd


In [11]:
%%file rmath.pyx

cimport rmath

def py_norm(double mu, double sd):
    return rmath.rnorm(mu, sd)

Overwriting rmath.pyx


In [12]:
%%file setup.py

from distutils.core import setup, Extension
from Cython.Build import cythonize
import numpy as np

ext = Extension("rmath",
                sources=["rmath.pyx"],
                include_dirs = ["."],
                library_dirs = ["."],
                libraries=["m", "Rmath"],
                extra_compile_args=["-w"])

setup(name = "RMath Funcs",
      ext_modules = cythonize(ext))

Overwriting setup.py


In [13]:
! python setup.py clean
! python setup.py -q build_ext --inplace

Compiling rmath.pyx because it changed.
[1/1] Cythonizing rmath.pyx
running clean
removing 'build/temp.macosx-10.5-x86_64-3.5' (and everything under it)
removing 'build'
In file included from rmath.c:262:
./Rmath.h:204:11: fatal error: 'R_ext/Boolean.h' file not found
# include <R_ext/Boolean.h>
          ^
1 error generated.
error: command 'gcc' failed with exit status 1


In [14]:
import rmath

rmath.rnom(10, 1)

ImportError: dlopen(/Users/cliburn/git/sta-663-2016/lectures/rmath.cpython-35m-darwin.so, 2): Symbol not found: _Rf_rnorm
  Referenced from: /Users/cliburn/git/sta-663-2016/lectures/rmath.cpython-35m-darwin.so
  Expected in: dynamic lookup
